In [1]:
import pandas as pd
import os
import re

# Define the path to the data folder and output path
data_path = "/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/ALL_STOCKMARKET_DATA_2020_24/IT_SECTOR_2020_24"
output_path = "/Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/DATA_OUTPUTS/CSV/ALL_YEAR_DATA_REFACTORED_CHATGPT_SEP_4.csv"

# Initialize an empty dataframe to store the concatenated data
all_year_data = pd.DataFrame()

# Load and concatenate data from all folders
folders = os.listdir(data_path)
if '.DS_Store' in folders:
    folders.remove('.DS_Store')

for folder in folders:
    for year in range(2020, 2025):
        year_path = os.path.join(data_path, folder, str(year))
        if os.path.exists(year_path):
            files = os.listdir(year_path)
            for file in files:
                file_path = os.path.join(year_path, file)
                data = pd.read_csv(file_path)

                # Extract ticker name from file name
                ticker_name = re.findall(r'(\w+)_\d{4}\.csv', file)[0]
                data['Ticker'] = ticker_name

                # Concatenate data
                all_year_data = pd.concat([all_year_data, data], ignore_index=True)

# Step 1: Calculate Daily Change in Price Percentage based on 5-day moving average
all_year_data['Daily_Change_In_Price'] = (all_year_data['Close'] - all_year_data['Close'].shift(1)) / all_year_data['Close'].shift(1)

# Calculate 5-day moving average
all_year_data['MA_5'] = all_year_data['Close'].rolling(window=5).mean().shift(1)

# Drop rows with missing moving averages
all_year_data.dropna(subset=['MA_5'], inplace=True)

# Step 2: Create binary columns for profit percentages (5%, 10%, ..., 100%)
thresholds = [5, 10, 25, 50, 100]
for threshold in thresholds:
    column_name = f'Profit_{threshold}_Percent'
    all_year_data[column_name] = all_year_data['Daily_Change_In_Price'].apply(lambda x: 1 if x >= threshold / 100 else 0)

# Step 3: One-Hot Encode the 'Index' and 'Ticker' columns
if 'Index' in all_year_data.columns:
    all_year_data = pd.get_dummies(all_year_data, columns=['Index'], prefix='Index')
if 'Ticker' in all_year_data.columns:
    all_year_data = pd.get_dummies(all_year_data, columns=['Ticker'], prefix='Ticker')

# Step 4: Convert 'Date' column to datetime and extract year, month, day
if 'Unnamed: 0' in all_year_data.columns:
    all_year_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
    all_year_data['Date'] = pd.to_datetime(all_year_data['Date'])

# Extract year, month, day as separate columns
all_year_data['Year'] = all_year_data['Date'].dt.year
all_year_data['Month'] = all_year_data['Date'].dt.month
all_year_data['Day'] = all_year_data['Date'].dt.day

# Drop the 'Date' column if not needed
all_year_data.drop(columns=['Date'], inplace=True)

# Step 5: Save the final DataFrame to the output path
all_year_data.to_csv(output_path, index=False)

print(f"Data has been successfully processed and saved to: {output_path}")


Data has been successfully processed and saved to: /Users/ronaldsheaks/Desktop/OSU AI Bootcamp/Group Project 2/GitHub Repo/Project_2_StockMrkt_ML/DATA_OUTPUTS/CSV/ALL_YEAR_DATA_REFACTORED_CHATGPT_SEP_4.csv
